# Assignment 2: Monte Carlo (MC) Methods in Finance

## Part I Option Valuation

## Part II Estimation of Sensitivities in MC

### Q1: Use bump-and-revalue method to estimate delta

## Part III Variance Reduction